In [8]:
import os
import time
import base64
import json
import requests
import sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementClickInterceptedException, ElementNotInteractableException
from selenium.webdriver.common.keys import Keys
from function1 import scroll_to_middle, send_text, click_element, set_date_field
from selenium.webdriver.common.action_chains import ActionChains

In [9]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options

firefox_options = Options()
firefox_options.add_argument("--start-maximized")

driver = webdriver.Firefox(options=firefox_options)
driver.maximize_window()

driver.get("https://www.mca.gov.in/content/mca/global/en/mca/llp-e-filling/Fillip.html")



In [10]:
with open("config_data.json", "r") as f:
    config_data = json.load(f)

In [13]:
# Income Source - fixing the potential issue with this line
time.sleep(2)
click_element(driver, css_selector='#guideContainer-rootPanel-panel-panel_358466187-panel-panel_copy_copy-panel-panel_copy-panel_copy-panel_969275493_copy-guidetextbox_2064455___widget')
time.sleep(1)
send_text(driver, css_selector='#guideContainer-rootPanel-panel-panel_358466187-panel-panel_copy_copy-panel-panel_copy-panel_copy-panel_969275493_copy-guidetextbox_2064455___widget',keys=config_data['form_data']['fields']['Income Source'])


In [15]:
            # Wait a little before interaction
time.sleep(2)
            # Locate the element first
element = driver.find_element(By.CSS_SELECTOR, '#guideContainer-rootPanel-panel-panel_358466187-panel-panel_copy_copy-panel-panel_copy-panel_copy-panel_969275493_copy-guidetextbox_2064455___widget')
            # Scroll to middle of viewport
driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", element)
            # Click the element
click_element(driver, css_selector='#guideContainer-rootPanel-panel-panel_358466187-panel-panel_copy_copy-panel-panel_copy-panel_copy-panel_969275493_copy-guidetextbox_2064455___widget')
            # Wait and send keys
time.sleep(1)
send_text(driver, css_selector='#guideContainer-rootPanel-panel-panel_358466187-panel-panel_copy_copy-panel-panel_copy-panel_copy-panel_969275493_copy-guidetextbox_2064455___widget', keys=config_data['form_data']['fields']['Income Source'])

In [50]:
import os
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.firefox import GeckoDriverManager

def validate_file_paths(file_paths):
    """Validate that all file paths exist."""
    for file_path in file_paths:
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")

def upload_nominee_identity_proofs(driver, config, max_wait=10):
    """
    Uploads 'Proof of Identity' files for nominees in 'bodies_corporate_nominee_no_din' section.
    Uses stable selectors to handle dynamic IDs and verifies uploads.
    """
    nominees = config.get('bodies_corporate_nominee_no_din', [])
    file_paths = [nominee.get("uploads", {}).get("identity_proof_path", "").strip() for nominee in nominees]
    file_paths = [path for path in file_paths if path]  # Filter out empty paths

    if not file_paths:
        print("[WARNING] No valid file paths provided in config.")
        return

    try:
        # Validate file paths
        validate_file_paths(file_paths)

        # Find all file input fields with uniquename="6dvIdentityProof"
        file_inputs = driver.find_elements(By.CSS_SELECTOR, 'input[uniquename="6dvIdentityProof"][type="file"]')
        print(f"[INFO] Found {len(file_inputs)} 'Proof of identity' file input fields.")

        if not file_inputs:
            raise Exception("No Proof of identity file inputs found.")

        # Check if enough files are provided
        if len(file_paths) < len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but {len(file_inputs)} fields found. Using available files.")

        # Upload files to the detected fields
        for i, (file_input, file_path) in enumerate(zip(file_inputs[:len(file_paths)], file_paths)):
            file_path = os.path.abspath(file_path)
            try:
                # Ensure the file input is interactable
                driver.execute_script("""
                    arguments[0].style.display = 'block';
                    arguments[0].style.visibility = 'visible';
                    arguments[0].style.opacity = 1;
                    arguments[0].removeAttribute('disabled');
                    arguments[0].removeAttribute('hidden');
                """, file_input)

                # Scroll to the element to ensure visibility
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", file_input)
                time.sleep(0.5)  # Brief pause for stability

                # Upload the file
                file_input.send_keys(file_path)
                print(f"[SUCCESS] Uploaded file to field {i+1}: {os.path.basename(file_path)}")

                # Verify the upload by checking the file list
                try:
                    container = file_input.find_element(By.XPATH, "./ancestor::div[contains(@class, 'fileUpload')]")
                    file_list = container.find_element(By.CSS_SELECTOR, "ul.guide-fu-fileItemList")
                    WebDriverWait(driver, max_wait).until(
                        EC.text_to_be_present_in_element(
                            (By.CSS_SELECTOR, "ul.guide-fu-fileItemList li"),
                            os.path.basename(file_path)
                        )
                    )
                    print(f"[VERIFIED] File appears in UI list for field {i+1}")
                except TimeoutException:
                    print(f"[WARNING] File may not have appeared in UI list for field {i+1}")
                except NoSuchElementException:
                    print(f"[WARNING] File list not found for field {i+1}")

                # Check for validation errors
                try:
                    error = container.find_element(By.CSS_SELECTOR, ".guideFieldError")
                    if error.is_displayed() and "This Field is a required field." in error.text:
                        print(f"[ERROR] Validation error in field {i+1}: {error.text}")
                except NoSuchElementException:
                    pass  # No error found, which is good

            except NoSuchElementException:
                print(f"[ERROR] File input not interactable for field {i+1}")
            except Exception as e:
                print(f"[ERROR] Failed to upload file to field {i+1}: {type(e).__name__} - {e}")

        if len(file_paths) > len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but only {len(file_inputs)} fields found. Excess files ignored.")

        print(f"[INFO] Completed uploading to {min(len(file_inputs), len(file_paths))} fields.")

    except FileNotFoundError as e:
        print(f"[ERROR] {e}")
    except Exception as e:
        print(f"[FATAL] Unexpected error: {type(e).__name__} - {e}")


try:
        with open("config_data.json", "r") as f:
            config_data = json.load(f)
except FileNotFoundError:
        print("[ERROR] config_data.json not found.")
except json.JSONDecodeError:
        print("[ERROR] Invalid JSON in config_data.json.")


In [51]:
# Upload files
upload_nominee_identity_proofs(driver, config_data)

[INFO] Found 2 'Proof of identity' file input fields.
[SUCCESS] Uploaded file to field 1: dummypdf.pdf
[VERIFIED] File appears in UI list for field 1
[SUCCESS] Uploaded file to field 2: dummypdf.pdf
[VERIFIED] File appears in UI list for field 2
[INFO] Completed uploading to 2 fields.


In [53]:
import os
import time
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from webdriver_manager.firefox import GeckoDriverManager

def upload_nominee_residential_proofs(driver, config, max_wait=10):
    """
    Uploads 'Residential proof' files for nominees in 'bodies_corporate_nominee_no_din' section.
    Uses stable selectors to handle dynamic IDs and verifies uploads.
    """
    nominees = config.get('bodies_corporate_nominee_no_din', [])
    file_paths = [nominee.get("uploads", {}).get("residential_proof_path", "").strip() for nominee in nominees]
    file_paths = [path for path in file_paths if path]  # Filter out empty paths

    if not file_paths:
        print("[WARNING] No valid file paths provided in config for Residential proof.")
        return

    try:
        # Validate file paths
        validate_file_paths(file_paths)

        # Find all file input fields with uniquename="6dvResidentialProof"
        file_inputs = driver.find_elements(By.CSS_SELECTOR, 'input[uniquename="6dvResidentialProof"][type="file"]')
        print(f"[INFO] Found {len(file_inputs)} 'Residential proof' file input fields.")

        if not file_inputs:
            raise Exception("No Residential proof file inputs found.")

        # Check if enough files are provided
        if len(file_paths) < len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but {len(file_inputs)} fields found. Using available files.")

        # Upload files to the detected fields
        for i, (file_input, file_path) in enumerate(zip(file_inputs[:len(file_paths)], file_paths)):
            file_path = os.path.abspath(file_path)
            try:
                # Ensure the file input is interactable
                driver.execute_script("""
                    arguments[0].style.display = 'block';
                    arguments[0].style.visibility = 'visible';
                    arguments[0].style.opacity = 1;
                    arguments[0].removeAttribute('disabled');
                    arguments[0].removeAttribute('hidden');
                """, file_input)

                # Scroll to the element to ensure visibility
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", file_input)
                time.sleep(0.5)  # Brief pause for stability

                # Upload the file
                file_input.send_keys(file_path)
                print(f"[SUCCESS] Uploaded file to field {i+1}: {os.path.basename(file_path)}")

                # Verify the upload by checking the file list
                try:
                    container = file_input.find_element(By.XPATH, "./ancestor::div[contains(@class, 'fileUpload')]")
                    file_list = container.find_element(By.CSS_SELECTOR, "ul.guide-fu-fileItemList")
                    WebDriverWait(driver, max_wait).until(
                        EC.text_to_be_present_in_element(
                            (By.CSS_SELECTOR, "ul.guide-fu-fileItemList li"),
                            os.path.basename(file_path)
                        )
                    )
                    print(f"[VERIFIED] File appears in UI list for field {i+1}")
                except TimeoutException:
                    print(f"[WARNING] File may not have appeared in UI list for field {i+1}")
                except NoSuchElementException:
                    print(f"[WARNING] File list not found for field {i+1}")

                # Check for validation errors
                try:
                    error = container.find_element(By.CSS_SELECTOR, ".guideFieldError")
                    if error.is_displayed() and "This Field is a required field." in error.text:
                        print(f"[ERROR] Validation error in field {i+1}: {error.text}")
                except NoSuchElementException:
                    pass  # No error found, which is good

            except NoSuchElementException:
                print(f"[ERROR] File input not interactable for field {i+1}")
            except Exception as e:
                print(f"[ERROR] Failed to upload file to field {i+1}: {type(e).__name__} - {e}")

        if len(file_paths) > len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but only {len(file_inputs)} fields found. Excess files ignored.")

        print(f"[INFO] Completed uploading to {min(len(file_inputs), len(file_paths))} fields.")

    except FileNotFoundError as e:
        print(f"[ERROR] {e}")
    except Exception as e:
        print(f"[FATAL] Unexpected error: {type(e).__name__} - {e}")
try:
        with open("config_data.json", "r") as f:
            config_data = json.load(f)
except FileNotFoundError:
        print("[ERROR] config_data.json not found.")
except json.JSONDecodeError:
        print("[ERROR] Invalid JSON in config_data.json.")

In [54]:

# Upload files
upload_nominee_residential_proofs(driver, config_data)

[INFO] Found 2 'Residential proof' file input fields.
[SUCCESS] Uploaded file to field 1: dummypdf.pdf
[VERIFIED] File appears in UI list for field 1
[SUCCESS] Uploaded file to field 2: dummypdf.pdf
[VERIFIED] File appears in UI list for field 2
[INFO] Completed uploading to 2 fields.


In [57]:

def upload_nominee_resolution_proofs(driver, config, max_wait=10):
    """
    Uploads 'Copy of resolution' files for nominees in 'bodies_corporate_nominee_no_din' section.
    Uses stable selectors to handle dynamic IDs and verifies uploads.
    """
    nominees = config.get('bodies_corporate_nominee_no_din', [])
    file_paths = [nominee.get("uploads", {}).get("resolution_copy_path", "").strip() for nominee in nominees]
    file_paths = [path for path in file_paths if path]  # Filter out empty paths

    if not file_paths:
        print("[WARNING] No valid file paths provided in config for Copy of resolution.")
        return

    try:
        # Validate file paths
        validate_file_paths(file_paths)

        # Find all file input fields with uniquename="6dvCopyOfResolution"
        file_inputs = driver.find_elements(By.CSS_SELECTOR, 'input[uniquename="6dvCopyOfResolution"][type="file"]')
        print(f"[INFO] Found {len(file_inputs)} 'Copy of resolution' file input fields.")

        if not file_inputs:
            raise Exception("No Copy of resolution file inputs found.")

        # Check if enough files are provided
        if len(file_paths) < len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but {len(file_inputs)} fields found. Using available files.")

        # Upload files to the detected fields
        for i, (file_input, file_path) in enumerate(zip(file_inputs[:len(file_paths)], file_paths)):
            file_path = os.path.abspath(file_path)
            try:
                # Ensure the file input is interactable
                driver.execute_script("""
                    arguments[0].style.display = 'block';
                    arguments[0].style.visibility = 'visible';
                    arguments[0].style.opacity = 1;
                    arguments[0].removeAttribute('disabled');
                    arguments[0].removeAttribute('hidden');
                """, file_input)

                # Scroll to the element to ensure visibility
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", file_input)
                time.sleep(0.5)  # Brief pause for stability

                # Upload the file
                file_input.send_keys(file_path)
                print(f"[SUCCESS] Uploaded file to field {i+1}: {os.path.basename(file_path)}")

                # Verify the upload by checking the file list
                try:
                    container = file_input.find_element(By.XPATH, "./ancestor::div[contains(@class, 'fileUpload')]")
                    file_list = container.find_element(By.CSS_SELECTOR, "ul.guide-fu-fileItemList")
                    WebDriverWait(driver, max_wait).until(
                        EC.text_to_be_present_in_element(
                            (By.CSS_SELECTOR, "ul.guide-fu-fileItemList li"),
                            os.path.basename(file_path)
                        )
                    )
                    print(f"[VERIFIED] File appears in UI list for field {i+1}")
                except TimeoutException:
                    print(f"[WARNING] File may not have appeared in UI list for field {i+1}")
                except NoSuchElementException:
                    print(f"[WARNING] File list not found for field {i+1}")

                # Check for validation errors
                try:
                    error = container.find_element(By.CSS_SELECTOR, ".guideFieldError")
                    if error.is_displayed() and "This Field is a required field." in error.text:
                        print(f"[ERROR] Validation error in field {i+1}: {error.text}")
                except NoSuchElementException:
                    pass  # No error found, which is good

            except NoSuchElementException:
                print(f"[ERROR] File input not interactable for field {i+1}")
            except Exception as e:
                print(f"[ERROR] Failed to upload file to field {i+1}: {type(e).__name__} - {e}")

        if len(file_paths) > len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but only {len(file_inputs)} fields found. Excess files ignored.")

        print(f"[INFO] Completed uploading to {min(len(file_inputs), len(file_paths))} fields.")

    except FileNotFoundError as e:
        print(f"[ERROR] {e}")
    except Exception as e:
        print(f"[FATAL] Unexpected error: {type(e).__name__} - {e}")


try:
        with open("config_data.json", "r") as f:
            config_data = json.load(f)
except FileNotFoundError:
        print("[ERROR] config_data.json not found.")
except json.JSONDecodeError:
        print("[ERROR] Invalid JSON in config_data.json.")
        

In [58]:
upload_nominee_resolution_proofs(driver, config_data)

[INFO] Found 2 'Copy of resolution' file input fields.
[SUCCESS] Uploaded file to field 1: dummypdf.pdf
[VERIFIED] File appears in UI list for field 1
[SUCCESS] Uploaded file to field 2: dummypdf.pdf
[VERIFIED] File appears in UI list for field 2
[INFO] Completed uploading to 2 fields.


In [63]:

def upload_nominee_resolution_proofs_form3c(driver, config, max_wait=10):
    """
    Uploads 'Copy of resolution' files for entities in 'bodies_corporate_with_din' section in Form 3 (C).
    Uses stable selectors to handle dynamic IDs and verifies uploads.
    """
    entities = config.get('bodies_corporate_with_din', [])
    file_paths = [entity.get("Copy of resolution", "").strip() for entity in entities]
    file_paths = [path for path in file_paths if path]  # Filter out empty paths

    if not file_paths:
        print("[WARNING] No valid file paths provided in config for Copy of resolution in Form 3 (C).")
        return

    try:
        # Validate file paths
        validate_file_paths(file_paths)

        # Find all file input fields with uniquename="BodyCorporate6c"
        file_inputs = driver.find_elements(By.CSS_SELECTOR, 'input[uniquename="BodyCorporate6c"][type="file"]')
        print(f"[INFO] Found {len(file_inputs)} 'Copy of resolution' file input fields in Form 3 (C).")

        if not file_inputs:
            raise Exception("No Copy of resolution file inputs found in Form 3 (C).")

        # Check if enough files are provided
        if len(file_paths) < len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but {len(file_inputs)} fields found. Using available files.")

        # Upload files to the detected fields
        for i, (file_input, file_path) in enumerate(zip(file_inputs[:len(file_paths)], file_paths)):
            file_path = os.path.abspath(file_path)
            try:
                # Ensure the file input is interactable
                driver.execute_script("""
                    arguments[0].style.display = 'block';
                    arguments[0].style.visibility = 'visible';
                    arguments[0].style.opacity = 1;
                    arguments[0].removeAttribute('disabled');
                    arguments[0].removeAttribute('hidden');
                """, file_input)

                # Scroll to the element to ensure visibility
                driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", file_input)
                time.sleep(0.5)  # Brief pause for stability

                # Upload the file
                file_input.send_keys(file_path)
                print(f"[SUCCESS] Uploaded file to field {i+1}: {os.path.basename(file_path)}")

                # Verify the upload by checking the file list
                try:
                    container = file_input.find_element(By.XPATH, "./ancestor::div[contains(@class, 'fileUpload')]")
                    file_list = container.find_element(By.CSS_SELECTOR, "ul.guide-fu-fileItemList")
                    WebDriverWait(driver, max_wait).until(
                        EC.text_to_be_present_in_element(
                            (By.CSS_SELECTOR, "ul.guide-fu-fileItemList li"),
                            os.path.basename(file_path)
                        )
                    )
                    print(f"[VERIFIED] File appears in UI list for field {i+1}")
                except TimeoutException:
                    print(f"[WARNING] File may not have appeared in UI list for field {i+1}")
                except NoSuchElementException:
                    print(f"[WARNING] File list not found for field {i+1}")

                # Check for validation errors
                try:
                    error = container.find_element(By.CSS_SELECTOR, ".guideFieldError")
                    if error.is_displayed() and "This Field is a required field." in error.text:
                        print(f"[ERROR] Validation error in field {i+1}: {error.text}")
                except NoSuchElementException:
                    pass  # No error found, which is good

            except NoSuchElementException:
                print(f"[ERROR] File input not interactable for field {i+1}")
            except Exception as e:
                print(f"[ERROR] Failed to upload file to field {i+1}: {type(e).__name__} - {e}")

        if len(file_paths) > len(file_inputs):
            print(f"[WARNING] {len(file_paths)} files provided, but only {len(file_inputs)} fields found. Excess files ignored.")

        print(f"[INFO] Completed uploading to {min(len(file_inputs), len(file_paths))} fields.")

    except FileNotFoundError as e:
        print(f"[ERROR] {e}")
    except Exception as e:
        print(f"[FATAL] Unexpected error: {type(e).__name__} - {e}")


                # Load configuration
try:
                with open("config_data.json", "r") as f:
                    config_data = json.load(f)
except FileNotFoundError:
                print("[ERROR] config_data.json not found.")
except json.JSONDecodeError:
                print("[ERROR] Invalid JSON in config_data.json.")

In [64]:
upload_nominee_resolution_proofs_form3c(driver, config_data)

[INFO] Found 2 'Copy of resolution' file input fields in Form 3 (C).
[SUCCESS] Uploaded file to field 1: dummypdf.pdf
[VERIFIED] File appears in UI list for field 1
[SUCCESS] Uploaded file to field 2: dummypdf.pdf
[VERIFIED] File appears in UI list for field 2
[INFO] Completed uploading to 2 fields.
